<a href="https://colab.research.google.com/github/shinasto/scikit-learn/blob/main/11_CrossValidation_and_GirdSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련 세트를 또 나누는 것. 이 데이터를 검증 세트(validation set) 라고 부릅니다.
# 훈련세트 60%, 검증세트 20%, 테스트 세트 20%

# 교차 검증
# 교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복
# 그 다음 이 점수를 평균하여 최종 검증 점수를 얻습니다.
# 5-폴드 교차 검증, 10-폴트 교차 검증 을 많이 사용함 (데이터의 80~90% 까지 훈련에 사용 가능)
# 훈련 세트를 n 부분으로 나눠서 교차 검증을 수행하는 것을 n-폴트 교차 검증이라 함.

import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [5]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, tet_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [8]:
# 교차 검증 함수(cross_validate)

from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
scores = cross_validate(dt, train_input, train_target)
print(scores)

# fit_time : 훈련시간
# score_time : 검증시간
# test_score : 검증 점수  => 모든 검증 점수 평균이 교차 검증의 최종 점수

{'fit_time': array([0.00922108, 0.0079329 , 0.00762463, 0.007411  , 0.00709963]), 'score_time': array([0.00090981, 0.00079226, 0.00076056, 0.0007515 , 0.00073051]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [9]:
# 교차 검증 점수 
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [11]:
# cross_validate() 는 훈련 세트를 섞어 폴트를 나누지 않음
# 그래서 앞서 train_test_split() 함수로 전체 데이터를 섞은 후 훈련 세트를 준비
# 교차 검증 할때 훈련 세트를 섞으려면 분한기를 지정해야함 (KFold)
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [12]:
# 만약 10-폴트 교차 검증을 수행하려면 다음과 같이 작성

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [14]:
# 하이퍼파라미터 튜닝
# 머신러닝 모델이 학습하는 파라미터를 모델 파라미터라고 부르고
# 반면 사용자가 지정해야만 하는 파라미터를 하이퍼파라미터라 함.
# 하이퍼파라미터를 튜닝하는 자동으로 수행하는 기술을 AutoML 이라고 부름

# 사이킷 런에서 제공하는 그리드 서치(Grid Search) 를 사용하면 복잡한 매개변수의 반복 과정을 직접 구현할 필요 없음.
# GridSearchCV 클래스는 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행함.

from sklearn.model_selection import GridSearchCV

# min_impurity_decrease 매개변수의 최적값을 찾아보자.
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

# 그리드 서치 객체는 결정 트리 모델 min_impurity_decrease 값을 바궈가면 총 5번 실행
gs.fit(train_input, train_target)

# GridSearchCV의 cv 매개변수 기본값은 5이므로 총 5x5=25 개의 모델을 훈련
# 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련함.
# 그 결과는 beat_estimator_ 속성에 저장
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [15]:
# best_params에 최적의 파라미터 저장
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [16]:
# 5번의 교차 검증으로 얻은 점수
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [19]:
# 이번에는 더 복잡한 매개변수 조합을 탐색해 보자.
# 매개변수: max_depth, min_samples_split, min_impurity_decrese 
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20 ,1),
          'min_samples_split': range(2, 100, 10)}

# 위 매개변수로 수행할 교차 검증 횟수는 9*15*10=1350, 1350*5(5-폴트 교차) = 6750 

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)
print(gs.best_params_)

#GridSearchCV 클래스를 사용하니 매개변수를 일일이 바꿔가면 교차 검증을 수행하지 않고 원하는 
# 매개 변수 값을 나열하면 자동으로 교차 검증을 수행해서 최상의 매개변수를 찾을 수 있음.

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [22]:
# 랜덤 서치
# 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포객체를 전달합니다.

from scipy.stats import uniform, randint

# radint 범위를 지정, rvs로 지정된 개수만큼 값을 뽑음
rgen = randint(0, 10)
rgen.rvs(10)

array([7, 7, 6, 0, 5, 5, 8, 6, 7, 5])

In [23]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([100,  94, 112,  86, 103, 103,  85, 112, 105, 100]))

In [24]:
# 0~1 사이의 실수 추출
ugen = uniform(0, 1) 
ugen.rvs(10)

array([0.50969902, 0.58477369, 0.02224619, 0.40327759, 0.34574416,
       0.60518626, 0.14676692, 0.75908952, 0.26077516, 0.72401745])

In [26]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)}

from sklearn.model_selection import RandomizedSearchCV
# n_iter : 샘플링 횟수
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

In [27]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [29]:
# 최고의 교차 검증 점수
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [31]:
# best_estimator_ 에 저장된 최종 모델로 테스트 세트의 성능 확인
dt = gs.best_estimator_
print(dt.score(test_input, tet_target))

0.86
